In [1]:
import os
import warnings
import numpy as np
import luigi
import emu.neuralynx_io as nlx
from tqdm import tqdm as tqdm
from emu.pdil.raw import Electrophysiology,Participant,get_data_manifest,points_to_choice
from emu.neuralynx_io import nev_as_records
from emu.nwb import nlx_to_nwb
from pynwb import TimeSeries, NWBFile,NWBHDF5IO
from pynwb.ecephys import ElectricalSeries
from pynwb.misc import AnnotationSeries
import pandas as pd
import datetime
import glob

In [2]:
all_files = get_data_manifest()

In [3]:
seeg_root = os.path.expanduser('~/.emu/pdil/pt_01/SEEG/raw/')

nev_path = os.path.expanduser('~/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.Events.nev')
ncs_paths = sorted(glob.glob(os.path.join(seeg_root,'PO_Day_04.*.ncs')))

# ncs = nlx.load_ncs()
print('nev path: ',nev_path)
print('ncs_paths:')
for p in ncs_paths:
    print(p)

nev path:  /home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.Events.nev
ncs_paths:
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC1.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC10.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC11.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC12.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC13.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC14.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC15.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC16.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC17.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC18.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC19.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC2.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC20.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC21.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_04.CSC22.ncs
/home/elijahc/.emu/pdil/pt_01/SEEG/

In [4]:
p = Participant(patient_id=1,raw_files=all_files,seeg_raw_path=seeg_root)

In [5]:
# Create a list of download tasks for the POD4 ncs files
all_ncs_tasks = list(p.cache_ncs())
d4_tasks = [t for t in all_ncs_tasks if 'PO_Day_04' in t.output().path]

# Create a list of download tasks for the POD4 nev files
d4_nev_task = [t for t in list(p.cache_nev()) if 'PO_Day_04.Events.nev' in t.output().path][0]

In [6]:
# Make sure the files are cached locally
luigi.build(d4_tasks+[d4_nev_task],workers=4)

DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632004419788, file_name=PO_Day_04.CSC1.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632004419788_PO_Day_04_CSC1_n_5df6429b44   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632004417388, file_name=PO_Day_04.CSC10.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632004417388_PO_Day_04_CSC10__70601f89a3   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632004414988, file_name=PO_Day_04.CSC11.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632004414988_PO_Day_04_CSC11__48f27f8992   has status 

INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632004369419_PO_Day_04_CSC31__459754a80b   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632004393419, file_name=PO_Day_04.CSC32.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632004393419_PO_Day_04_CSC32__2e9875888f   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632004380219, file_name=PO_Day_04.CSC33.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632004380219_PO_Day_04_CSC33__106750ae6c   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=631983668328, file_name=PO_Day_04.CSC34.ncs, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is c

INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632004576221_PO_Day_04_CSC9_n_5a0c196a53   has status   DONE
DEBUG: Checking if NLXRaw(data_root=/home/elijahc/.emu/, file_id=632004571421, file_name=PO_Day_04.Events.nev, save_to=/home/elijahc/.emu/pdil/pt_01/SEEG/raw/, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   NLXRaw__home_elijahc__e_632004571421_PO_Day_04_Events_9bfcf0a1e6   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 4 processes
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=004785645, workers=4, host=jatlab-remote, username=elijahc, pid=53291) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 51 tasks of which:
* 51 complete ones were encountered:
    - 51 NLXRaw(data_root=/home/elijahc/.emu/, file_id=631983667128, file_name=PO_Day_04.CSC44.ncs, save_to=/home/elijahc/.em

True

In [7]:
nwb = p.create_nwb(nev_path,ncs_paths,'Patient 1 | Post-op Day 4')

adding to nwb: 0it [00:00, ?it/s]
compressing channels:   0%|          | 0/49 [00:00<?, ?it/s]


4  blocks




compressing channels:  14%|█▍        | 7/49 [00:07<00:44,  1.06s/it]

Error loading PO_Day_04.CSC17.ncs
skipping...



compressing channels: 100%|██████████| 49/49 [00:50<00:00,  1.03s/it]
adding to nwb: 49it [00:50,  1.04s/it]
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629615908637, file_name=blockNum_3_computerTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629615908637_blockNum_3_compu_3484a4f2e7   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629620274728, file_name=blockNum_4_humanTT_defect_taskoutput.mat, save_to=/home/elijahc/.emu/pdil/pt_01/Behavior/raw, overwrite=False, study=pdil, patient_id=1) is complete
INFO: Informed scheduler that task   BehaviorRaw__home_elijahc__e_629620274728_blockNum_4_human_73d59337a6   has status   DONE
DEBUG: Checking if BehaviorRaw(data_root=/home/elijahc/.emu/, file_id=629611713944, file_name=blockNum_5_humanTT_coop_taskoutput.mat, save_to=/home

Failed to load:  channel_15


In [8]:
nwb

root pynwb.file.NWBFile at 0x140101569419528
Fields:
  acquisition: {
    channel_0 <class 'pynwb.base.TimeSeries'>,
    channel_1 <class 'pynwb.base.TimeSeries'>,
    channel_10 <class 'pynwb.base.TimeSeries'>,
    channel_11 <class 'pynwb.base.TimeSeries'>,
    channel_12 <class 'pynwb.base.TimeSeries'>,
    channel_13 <class 'pynwb.base.TimeSeries'>,
    channel_14 <class 'pynwb.base.TimeSeries'>,
    channel_15 <class 'pynwb.base.TimeSeries'>,
    channel_17 <class 'pynwb.base.TimeSeries'>,
    channel_18 <class 'pynwb.base.TimeSeries'>,
    channel_19 <class 'pynwb.base.TimeSeries'>,
    channel_2 <class 'pynwb.base.TimeSeries'>,
    channel_20 <class 'pynwb.base.TimeSeries'>,
    channel_21 <class 'pynwb.base.TimeSeries'>,
    channel_22 <class 'pynwb.base.TimeSeries'>,
    channel_23 <class 'pynwb.base.TimeSeries'>,
    channel_24 <class 'pynwb.base.TimeSeries'>,
    channel_25 <class 'pynwb.base.TimeSeries'>,
    channel_26 <class 'pynwb.base.TimeSeries'>,
    channel_27 <class

In [9]:
nwb.acquisition['ttl']

ttl pynwb.misc.AnnotationSeries at 0x140098031971464
Fields:
  comments: no comments
  conversion: 1.0
  data: ['block_start' 'block_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'block_start' 'block_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'block_start'
 'block_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'block_start' 'block_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_start' 'trial_start' 'trial_start'
 'trial_start' 'trial_start' 'trial_s

In [10]:
nwb.trials.to_dataframe().head()

,start_time,stop_time,outcome
id,,,
0,208.704119,225.619469,C-C
1,225.637112,240.257794,C-C
2,240.270138,254.523553,D-C
3,254.536538,268.636368,C-D
4,268.652939,303.003493,D-C


In [11]:
with NWBHDF5IO(os.path.join(seeg_root,'..','processed','PO_Day_04.nwb'),'w') as io:
    io.write(nwb)